In [1]:
############################# initialize parameter ####################################
dic = {'START':0,'B-positive':1,'I-positive':2,'B-neutral':3,'I-neutral':4,'B-negative':5,'I-negative':6,'O':7,'STOP':8}
l = ['START', 'B-positive', 'I-positive', 'B-neutral', 'I-neutral', 'B-negative', 'I-negative', 'O', 'STOP']
# store emission parameters
""" data structure: tuple + dictionary
    tuple: sentiment e.g.: 'B-positive' is second dic in this tuple
    dictionary: key: words, value: # of times it appears with this sentiment
                e.g.: {'my': 12} means that 'my' word appears 12 times in the sentiment 'o'
"""
e_param = ({},{},{},{},{},{},{},{},{})
# store transition parameters
"""data structure: matrix 9x9
   row: y(i) (from START to STOP), column: y(i+1) (from START to STOP)
   to record # of transsition from y(i) to y(i+1)
   e.g.: (START->O): matrix[0][7] ([dic][dic])
"""
t_param = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
           [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
# count # of sentiment appears+1
"""data structure: vector [9] (from START to STOP)
      vector[i] = sum(t_param[i])+1
"""
count = [0,0,0,0,0,0,0,0,0]

## read and parse file
train_file = open('train_EN','r', encoding='UTF-8') ###################need arg[0]!!!!!!!!!!!!!!
yi='START'
for r in train_file:
    try:
        var, sentiment = r.split()
        position = dic[sentiment]
        # update e_param
        temp_dic = e_param[position]
        if (var in temp_dic):
            temp_dic[var] = temp_dic[var]+1
        else:
            temp_dic[var] = 1
        # update t_param
        pre_position = dic[yi]
        t_param[pre_position][position] = t_param[pre_position][position]+1
        yi = sentiment
    except:
        # meaning the end of a sentence: x->STOP
        pre_position = dic[yi]
        t_param[pre_position][7] = t_param[pre_position][7]+1
        yi = 'START'
# get count(yi)+1
for i in range(0,9):
    temp_sum = 0
    for j in range(0,9):
        temp_sum = temp_sum+t_param[i][j]
    count[i] = temp_sum+1
# print (t_param)
#print (e_param)
# print (count)

############################### Part 3 ######################################
def viterbi(X):
    Y = []
    scores = []
    parents = []
    pi = []
    # calculate START -> 1
    temp_score = []
    temp_parent = []
    for i in range(1,8):
        #calculate emission first
        if (X[0] in e_param[i]):
            b = e_param[i][X[0]]/count[i]
        else:
            b = 1/count[i]
        score = t_param[0][i]*b
        temp_score.append(score)
    scores.append(temp_score)
    parents.append([0,0,0,0,0,0,0])
    # calculate 1 -> len(X)
    for i in range(1, len(X)):
        pre_score = temp_score
        temp_score = []
        temp_parent = []
        for j in range(1,8): # current state: j
            #calculate emission first
            if (X[i] in e_param[j]):
                b = e_param[j][X[i]]/count[j]
            else:
                b = 1/count[j]
            maxScore = -1000000
            maxParent = 0
            for k in range(1,8): # previous state: k
                current_Score = pre_score[k-1]*t_param[k][j]*b
                if(current_Score>maxScore):
                    maxScore = current_Score
                    maxParent = k
            temp_score.append(maxScore)
            temp_parent.append(maxParent)
        scores.append(temp_score)
        parents.append(temp_parent)
    # calculate len(X) -> STOP
    pre_score = temp_score
    temp_score = []
    temp_parent = []
    for i in range(1,8):
        maxScore = -1000000
        maxParent = 0
        for k in range(1,8): # previous state: k
            current_Score = pre_score[k-1]*t_param[k][j]*b
            if(current_Score>maxScore):
                maxScore = current_Score
                maxParent = k
    temp_score.append(maxScore)
    temp_parent.append(maxParent)
    scores.append(temp_score)
    parents.append(temp_parent)
    
    #print(scores)
    #print (parents)
    # backtracking
    parent = 1
    for i in range(len(parents)-1,0,-1):
        parent = parents[i][parent-1]
        Y.insert(0, l[parent])
    #print(Y)
    return Y

#viterbi(['New','Year',','])

dev_file = open('dev_EN.in','r', encoding='UTF-8')   #################################arg[1]!!!!!!!!!!!!!!
out_file = open('pred.out','w')
X = []
for r in dev_file:
    r = r.strip()
    if (r==''):
        #end of a sequence
        Y = viterbi(X)
        for i in range(0,len(X)):
            out_file.write(''+X[i]+" "+Y[i]+'\n')
        out_file.write('\n')
        X=[]
        continue
    X.append(r)
#####################calculate precisona and recell later... :D

